# SQL Server Ground to Cloud Security Workshop

_Last Updated: 25 July 2022_

In [this workshop](https://github.com/David-Seis/SecureYourAzureData) you'll cover the basics of securing SQL Server installations and databases, from on-premises systems to Microsoft Azure deployments.

_Note that this Notebook is an example only; and should not be used in production. It should be evaluated as a training implement only, and you should use these scripts and statements in a read-only fashion on a test system not connected to your production environment._

# Module 2 - SQL Server Security Basics

Activities from the [SQL Server Security Course](https://github.com/David-Seis/SecureYourAzureData), for [Module 2](https://github.com/David-Seis/SecureYourAzureData/blob/main/SQLSecurity/02%2520-%2520SQLServerSecurityBasics.md).

In [1]:
/* 
Module 2, Activity 1
Create Users blah blah blah
 */

SELECT * FROM sys.server_principals;
GO

(21 rows affected)

Total execution time: 00:00:00.146

name,principal_id,sid,type,type_desc,is_disabled,create_date,modify_date,default_database_name,default_language_name,credential_id,owning_principal_id,is_fixed_role
sa,1,0x01,S,SQL_LOGIN,0,2003-04-08 09:10:35.460,2021-10-31 22:07:19.670,master,us_english,NULL,NULL,0
public,2,0x02,R,SERVER_ROLE,0,2009-04-13 12:59:06.030,2009-04-13 12:59:06.030,NULL,NULL,NULL,1,0
sysadmin,3,0x03,R,SERVER_ROLE,0,2009-04-13 12:59:06.030,2009-04-13 12:59:06.030,NULL,NULL,NULL,1,1
securityadmin,4,0x04,R,SERVER_ROLE,0,2009-04-13 12:59:06.030,2009-04-13 12:59:06.030,NULL,NULL,NULL,1,1
serveradmin,5,0x05,R,SERVER_ROLE,0,2009-04-13 12:59:06.030,2009-04-13 12:59:06.030,NULL,NULL,NULL,1,1
setupadmin,6,0x06,R,SERVER_ROLE,0,2009-04-13 12:59:06.030,2009-04-13 12:59:06.030,NULL,NULL,NULL,1,1
processadmin,7,0x07,R,SERVER_ROLE,0,2009-04-13 12:59:06.030,2009-04-13 12:59:06.030,NULL,NULL,NULL,1,1
diskadmin,8,0x08,R,SERVER_ROLE,0,2009-04-13 12:59:06.030,2009-04-13 12:59:06.030,NULL,NULL,NULL,1,1
dbcreator,9,0x09,R,SERVER_ROLE,0,2009-04-13 12:59:06.030,2009-04-13 12:59:06.030,NULL,NULL,NULL,1,1
bulkadmin,10,0x0A,R,SERVER_ROLE,0,2009-04-13 12:59:06.030,2009-04-13 12:59:06.030,NULL,NULL,NULL,1,1
